# Продвинутый Python, семинар 11

**Лектор:** Петров Тимур

**Семинаристы:** Бузаев Федор, Дешеулин Олег, Коган Александра, Васина Олеся, Садуллаев Музаффар

**NB:** код внутри колаба не запустится

## Продолжаем мучать телегу

### Шаг 5. Инлайн-кнопки

Инлайн-кнопки - это кнопки, которые находятся в самом сообщении. В отличии от обычных кнопок, они постоянно находятся в чате, и по ним можно тыкнуть всегда.

Бывает три вида таких кнопок:

* Кнопки-ссылки

* Callback кнопки

* Switch кнопки (рассмотрим отдельно)

Самые простые кнопки - это ссылки. Работают они примерно так:

In [ ]:
from aiogram import Bot, Dispatcher, html, F, Router
from aiogram.filters import CommandStart, Command
from aiogram.types import Message, KeyboardButton, ReplyKeyboardMarkup, KeyboardButtonPollType, InlineKeyboardButton
from aiogram.utils.keyboard import InlineKeyboardBuilder

@text_router.message(Command("inline_url"))
async def cmd_inline_url(message: Message, bot: Bot):
    builder = InlineKeyboardBuilder()
    builder.row(InlineKeyboardButton(
        text="GitHub", url="https://github.com")
    )
    builder.row(InlineKeyboardButton(
        text="Оф. канал Telegram",
        url="tg://resolve?domain=telegram")
    )
    await message.answer(
        'Выберите ссылку',
        reply_markup=builder.as_markup(),
    )

По сути все то же самое, что и с обычными кнопками, единственное, что у нас все меняется на inline. В целом никаких сложностей нет.

Заострим внимание на callback кнопки. Что это такое?

Это кнопки, которые при нажатии отправляют в сам бот послание, которое мы можем дальше обрабатывать внутри (то есть мы не посылаем сообщение, а сразу же отправляем в диспетчер событие)

И для подобного callback у нас появляется новый хендлер - callback_query

Давайте для примера сделаем кнопку, которая будет генерить и выдавать рандомное число:

In [ ]:
from aiogram.filters import CommandStart, Command
from aiogram.types import Message, KeyboardButton, ReplyKeyboardMarkup, KeyboardButtonPollType, InlineKeyboardButton, CallbackQuery
from numpy.random import randint
from aiogram.utils.keyboard import InlineKeyboardBuilder


## Добавляем кнопку с callback
@text_router.message(Command("random_number"))
async def cmd_random(message: Message):
    builder = InlineKeyboardBuilder()
    builder.add(InlineKeyboardButton(
        text="Тык",
        callback_data="random_number")
    )
    await message.answer(
        "Генерируй число!",
        reply_markup=builder.as_markup()
    )

# Обрабатываем сам callback
@text_router.callback_query(F.data == "random_number") #обратите внимание, тут смотрим не сообщение, а data, которую передали
async def send_random_value(callback: CallbackQuery):
    await callback.message.reply(str(randint(1, 100)))

Запускаем, тыкаем... Работает! Другое дело, что он как будто еще что-то ожидает (как будто что-то не кончилось, сама кнопка мигает)

Почему так? Потому что в общем случае callback_query ожидает ответ, что все закончилось, а мы пока этого не передали. Что надо добавить? А вот это:

In [ ]:
from aiogram.filters import CommandStart, Command
from aiogram.types import Message, KeyboardButton, ReplyKeyboardMarkup, KeyboardButtonPollType, InlineKeyboardButton, CallbackQuery
from numpy.random import randint
from aiogram.utils.keyboard import InlineKeyboardBuilder


## Добавляем кнопку с callback
@text_router.message(Command("random_number"))
async def cmd_random(message: Message):
    builder = InlineKeyboardBuilder()
    builder.add(InlineKeyboardButton(
        text="Тык",
        callback_data="random_number")
    )
    await message.answer(
        "Генерируй число!",
        reply_markup=builder.as_markup()
    )

# Обрабатываем сам callback
@text_router.callback_query(F.data == "random_number") #обратите внимание, тут смотрим не сообщение, а data, которую передали
async def send_random_value(callback: CallbackQuery):
    await callback.message.reply(str(randint(1, 100)))
    await callback.answer()

Таким образом и даже ожидать ничего не нужно, запрос закрылся! Отлично, научились пользоваться callback кнопками

### Шаг 6. Конечные автоматы

В теории алгоритмов есть такая вещь, как конечные автоматы (Finite State Machine, FSM). Что это такое?

Это набор состояний, переходов и условий для переходов в новое состояние (вот [тут](https://habr.com/ru/articles/358304/) можно почитать чуть подробнее). Что нас здесь интересует с точки зрения Телеграма?

Если задуматься, множество процессов есть конечный автомат. Давайте на примере какого-нибудь дейтинга:

1. Регистрация
2. Ввод почты
3. Ввод имени
4. Загрузка фото
5. Загрузка bio
6. Profit

Каждый этап у вас идет один за другим. Если не считать, что все можно скипнуть, то пока вы не выполните каждый из этапов (состояние), то вы не перейдете в следующий (условие перехода). Соответственно, для ботов можно задавать такую же логику работы

Для этого будем с нуля создавать бота для заказа пиццы!

Так как у нас тут домашняя поделка, то пока что не будем думать про БД, а сделаем все достаточно банально в виде списков. Допустим, что у нас есть следующая конфигурация:

In [ ]:
available_pizza = ["Маргарита", "Пепперони", "Ветчина и сыр"]
available_sizes = ["Маленькая (25см)", "Средняя (30см)", "Большая (40см)"]

Что мы хотим видеть от бота? Хотим видеть следующую структуру:

1. Выбор пиццы
2. Выбор размера

Размер нельзя выбрать до выбора пиццы, оформить заказ нельзя без выбора обоих опций.

Для того, чтобы создать конечный автомат, нам нужны состояния. В нашем случае состояния понятны:

1. Выбрана пицца
2. Выбран размер

Пишем их с помощью инструментов aiogram (называется State)

In [ ]:
from aiogram.fsm.state import StatesGroup, State

class OrderFood(StatesGroup): #задаем класс для наших состояний
    choosing_food_name = State() # выбор пиццы
    choosing_food_size = State() # выбор размера

In [ ]:
@command_router.message(StateFilter(None), Command("order")) # Добавляем фильтр на состояние (Пустое)
async def cmd_food(message: Message, state: FSMContext):
    row = [KeyboardButton(text=item) for item in available_pizza]
    kb = ReplyKeyboardMarkup(keyboard=[row], resize_keyboard=True)
    await message.answer(
        text="Выберите блюдо:",
        reply_markup=kb
    )
    await state.set_state(OrderFood.choosing_food_name) #ставим ожидание выбора

Следующий шаг - отловить выбор еды:

In [ ]:
@command_router.message(
    OrderFood.choosing_food_name,
    F.text.in_(available_pizza)
)
async def food_chosen(message: Message, state: FSMContext):
    row = [KeyboardButton(text=item) for item in available_sizes]
    kb = ReplyKeyboardMarkup(keyboard=[row], resize_keyboard=True)
    await state.update_data(chosen_food=message.text) # апдейт по состоянию
    await message.answer(
        text="Спасибо. Теперь, пожалуйста, выберите размер порции:",
        reply_markup=kb
    )
    await state.set_state(OrderFood.choosing_food_size) # следующее состояние

Попробуем запустить. Что у нас получится? Вроде все как и должно быть

Но что будет, если пользователь попробует сам ввести и у него не получится ввести правильно? Правильно - ничего. Для этого можно добавить отдельный хендлер для отработки (опять-таки, используя текущий State)

Осталось добавить еще один шаг - докинуть общий заказ. Как нам выцепить всю прошлую инфу из диалога? А вот так:



In [ ]:
@command_router.message(OrderFood.choosing_food_size, F.text.in_(available_sizes))
async def food_size_chosen(message: Message, state: FSMContext):
    user_data = await state.get_data() # получаем всю скопленную информацию
    await message.answer(
        text=f"Вы выбрали {user_data['chosen_food']}, {message.text}", # тут ее можно использовать!
        reply_markup=ReplyKeyboardRemove() #Убираем клаву
    )
    await state.clear() #очищаем стейт

Ну а теперь последнее, что нам нужно сделать: обработать отмену (вдруг пользователь решил что-нибудь да поменять в заказе, правда ведь?)

In [ ]:
@command_router.message(StateFilter(None), Command(commands=["cancel"])) # отмена в самом начале
@command_router.message(default_state, F.text.lower() == "отмена") #default_state == пустой State
async def cmd_cancel_no_state(message: Message, state: FSMContext):
    await state.set_data({}) # убираем данные
    await message.answer(
        text="Нечего отменять",
        reply_markup=ReplyKeyboardRemove()
    )

@command_router.message(Command(commands=["cancel"]))
@command_router.message(F.text.lower() == "отмена")
async def cmd_cancel(message: Message, state: FSMContext):
    await state.clear() # чистим полностью
    await message.answer(
        text="Действие отменено",
        reply_markup=ReplyKeyboardRemove()
    )

Ну отлично, мы с вами научились делать диалоговые системы с каким никаким запоминанием ответов, победа!

### Шаг 7. Инлайн-режим

Что такое инлайн-режим?

Давайте на примере: если в телеге ввести @gif, то у вас всплывают гифки и поиск по ним. Собственно говоря, это и есть инлайн-режим (поиск внутри строки)

Это удобно для поиска (например, можно использовать в качестве поиска по функциям внутри бота).

Как это работает? Когда вы вводите название бота, то вызывается так называемый InlineQuery внутри бота, который можно обрабатывать.

Чтобы инлайн-режим у бота вообще работал, его вначале надо включить в самой телеге. Для этого заходим в BotFather (/setinline -> выбираем бота -> пишем placeholder)

Что же, давайте с вами на примере попробуем добавить inline query для нашего бота, чтобы подтянуть красивое инфо (добавим тот же самый order):

In [ ]:
@inline_router.inline_query()
async def show_user_links(inline_query: InlineQuery):
    commands = [r"/start", r"/order"]
    descriptions = ["Инфо", "Сделать заказ"]

    results = []
    for i in range(len(commands)):
        results.append(InlineQueryResultArticle(
            id=str(i),
            title=commands[i],
            description=descriptions[i],
            input_message_content=InputTextMessageContent(message_text=commands[i])
        ))
    await inline_query.answer(results)

Что тут происходит?

При выхове бота у нас создается InlineQuery, который как раз и прокидывается (так как хотим иметь что угодно, то никаких фильтров не добавляем)

После этого мы создаем массив из ответов, что он из себя представляет:

* InlineQueryResultArticle - наш объект для отображения

Внутри объекта должно быть несколько значений:

* id - id объекта (должен был уникальным)

* title - название

* description - описание

* input_message_content - ответ, который происходит при клике (в нашем случае просто отправляем команду)

В целом все, ура, мы собрали даже инлайн-режим для нашего простенького бота!

### Шаг 8. Собираем теперь воедино в красивую картину!

Теперь со всей информацией, которую мы получили, мы можем прямо-таки собрать красивого бота!

Что нам нужно добавить?

* Просмотр заказа и добавление нескольких товаров

* Отмена заказа

* Переход назад, если человек решил выбрать другую пиццу

* Кнопочка оплатить (просто поставим заглушку)

Итоговый результат:

text_commands:

In [ ]:
from aiogram import F, Router
from aiogram.types import Message
from aiogram.fsm.context import FSMContext
from handlers.commands import order
from aiogram.enums import ParseMode

text_router = Router()

@text_router.message(F.text.lower() == 'что это')
async def cmd_faq(message: Message):
    text = '''
    Привет! Это бот для заказа пиццы из нашей очень популярной пиццерии! 🦜
    Ты можешь сделать заказ с помощью команды /order
    '''
    await message.answer(text)

@text_router.message(F.text.lower() == 'посмотреть заказ')
async def cmd_lookup(message: Message):
    if len(order) == 0:
        await message.answer("В корзине пусто 😭", parse_mode=ParseMode.HTML)
    else:
        text = "Вот ваш заказ: \n"
        for i in order:
            text += f"<b> {i} </b> \n"
        await message.answer(text, parse_mode=ParseMode.HTML)

@text_router.message(F.text == 'Отменить заказ')
async def cmd_cancel_all(message: Message, state: FSMContext):
    order.clear()
    await state.clear()
    await message.answer("Ваш заказ отменен")

@text_router.message(F.text.lower() == 'оплата')
async def cmd_cancel_all(message: Message, state: FSMContext):
    order.clear()
    await state.clear()
    await message.answer("Отправили ваш заказ, ожидайте!")

@text_router.message()
async def cmd_dont_know(message: Message):
    text = '''
    Простите, пожалуйста, не понимаю, что вы хотите сказать 😭
    '''
    await message.answer(text)

inline_mode

In [ ]:
from aiogram import Router
from aiogram.types import InlineQuery, InlineQueryResultArticle, InputTextMessageContent


inline_router = Router()

@inline_router.inline_query()
async def show_user_links(inline_query: InlineQuery):
    commands = [r"/start", r"/order"]
    descriptions = ["Инфо", "Сделать заказ"]

    results = []
    for i in range(len(commands)):
        results.append(InlineQueryResultArticle(
            id=str(i),
            title=commands[i],
            description=descriptions[i],
            input_message_content=InputTextMessageContent(message_text=commands[i])
        ))
    await inline_query.answer(results)

commands

In [ ]:
from aiogram import F, Router
from aiogram.filters import CommandStart, Command, StateFilter
from aiogram.types import Message, KeyboardButton, ReplyKeyboardMarkup, ReplyKeyboardRemove
from aiogram.fsm.state import default_state
from aiogram.fsm.context import FSMContext
from states.order_state import OrderFood

order = []
available_pizza = ["Маргарита", "Пепперони", "Ветчина и сыр"]
available_sizes = ["Маленькая (25см)", "Средняя (30см)", "Большая (40см)"]

command_router = Router()

@command_router.message(CommandStart())
async def cmd_hello(message: Message):
    await message.answer("Добро пожаловать в ресторан PyPizza! Здесь вы можете оформить свой заказ")
    await message.answer(r"Оформите заказ с помощью команды /order")

@command_router.message(StateFilter(None), Command("order"))
@command_router.message(StateFilter(None), F.text.lower().contains("добавить"))
async def cmd_food(message: Message, state: FSMContext):
    order = []
    row = [KeyboardButton(text=item) for item in available_pizza]
    kb = ReplyKeyboardMarkup(keyboard=[row], resize_keyboard=True)
    await message.answer(
        text="Выберите блюдо:",
        reply_markup=kb
    )
    await state.set_state(OrderFood.choosing_food_name)

@command_router.message(
    OrderFood.choosing_food_name,
    F.text.in_(available_pizza)
)
async def food_chosen(message: Message, state: FSMContext):
    row = [KeyboardButton(text=item) for item in available_sizes]
    row_1 = [KeyboardButton(text="Вернуться назад")]
    kb = ReplyKeyboardMarkup(keyboard=[row, row_1], resize_keyboard=True)
    await state.update_data(chosen_food=message.text)
    await message.answer(
        text="Спасибо. Теперь, пожалуйста, выберите размер порции:",
        reply_markup=kb
    )
    await state.set_state(OrderFood.choosing_food_size)

@command_router.message(OrderFood.choosing_food_size, F.text.lower().contains("назад"))
async def food_choose_back(message: Message, state: FSMContext):
    await state.clear()
    row = [KeyboardButton(text=item) for item in available_pizza]
    kb = ReplyKeyboardMarkup(keyboard=[row], resize_keyboard=True)
    await message.answer(
        text=f"Возвращаю на выбор пиццы",
        reply_markup=kb
    )
    await state.set_state(OrderFood.choosing_food_name)

@command_router.message(OrderFood.choosing_food_size, F.text.in_(available_sizes))
async def food_size_chosen(message: Message, state: FSMContext):
    user_data = await state.get_data()
    row = [KeyboardButton(text="Добавить товар"), KeyboardButton(text="Оплата")]
    row_1 = [KeyboardButton(text="Посмотреть заказ"), KeyboardButton(text='Отменить заказ')]
    kb = ReplyKeyboardMarkup(keyboard=[row, row_1], resize_keyboard=True)
    order.append(f"{user_data['chosen_food']}, {message.text}")
    await message.answer(
        text=f"Вы выбрали {user_data['chosen_food']}, {message.text}",
        reply_markup=kb
    )
    await state.clear()

@command_router.message(StateFilter(None), Command(commands=["cancel"]))
@command_router.message(default_state, F.text.lower() == "отмена")
async def cmd_cancel_no_state(message: Message, state: FSMContext):
    await state.set_data({})
    await message.answer(
        text="Нечего отменять",
        reply_markup=ReplyKeyboardRemove()
    )

@command_router.message(Command(commands=["cancel"]))
@command_router.message(F.text.lower() == "отмена")
async def cmd_cancel(message: Message, state: FSMContext):
    await state.clear()
    await message.answer(
        text="Действие отменено",
        reply_markup=ReplyKeyboardRemove()
    )